# Phase 2
Analysing AML microarray.

## 1. Prepairing Data

### Installing Packages

In [ ]:
!pip uninstall rpy2 -y
!pip install rpy2==3.5.1
%load_ext rpy2.ipython

In [ ]:
%%R
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager", repos='http://cran.us.r-project.org')
BiocManager::install()
BiocManager::install("GEOquery")

install.packages("limma")
install.packages("umap")
install.packages("maptools")
install.packages("corrplot")
install.packages("pheatmap")
install.packages("reshape2")
install.packages("mvtnorm")
install.packages("Rtsne")
install.packages("M3C")
install.packages("MASS")
install.packages("ggpubr")
install.packages(c("vegan", "ecodist", "labdsv", "ape", "ade4", "smacof"))
install.packages("fastICA")


### Libraries

In [ ]:
%%R

library(Biobase)
library(GEOquery)
library(limma)
library(umap)
library(maptools)
library(tidyr)
library(dplyr)
library(ggplot2)
library(devtools)
install_github("vqv/ggbiplot")
library(ggbiplot)
library(data.table)
library(corrplot)
library(pheatmap)
library(reshape2)
library(plyr)
library(MASS)
library(mvtnorm)
library(Rtsne)
library("ggpubr")
library(vegan)
library(ecodist)
library(labdsv)
library(ape)
library(ade4)
library(fastICA)
library(smacof)

### Loading Data

In [4]:
%%R

gds <- getGEO("GSE48558", GSEMatrix = TRUE, AnnotGPL=TRUE)
show(gds)
gds48558 <- gds[[1]]

fvarLabels(gds48558) <- make.names(fvarLabels(gds48558))

# group membership for all samples
gsms <- paste0("1111111111111XXXXXXXXXXXXXXXXXXXXXXXXXXX0XXX0XXXXX",
               "XXXXXXXXXXXXXXXXXX0X0XXX0X0000X0XX00XX00X0X0X0X0X0",
               "XXX0XXX0XXXXXXXXXXXXXXXXXXXXXXXXXXXXX0000000110111",
               "00000000000000000000")
sml <- strsplit(gsms, split="")[[1]]

sel <- which(sml != "X")
sml <- sml[sel]
gds48558 <- gds48558[ ,sel]

gs <- factor(sml)
groups <- make.names(c("Healthy", "AML"))
levels(gs) <- groups
gds48558$group <- gs

$GSE48558_series_matrix.txt.gz
ExpressionSet (storageMode: lockedEnvironment)
assayData: 32321 features, 170 samples 
  element names: exprs 
protocolData: none
phenoData
  sampleNames: GSM1180750 GSM1180751 ... GSM1180919 (170 total)
  varLabels: title geo_accession ... phenotype:ch1 (32 total)
  varMetadata: labelDescription
featureData
  featureNames: 7892501 7892502 ... 8180418 (32321 total)
  fvarLabels: ID Gene title ... GO:Component ID (21 total)
  fvarMetadata: Column Description labelDescription
experimentData: use 'experimentData(object)'
  pubMedIds: 23836560 
Annotation: GPL6244 



In [5]:
%%R

expr <- exprs(gds48558)
features <- fData(gds48558)
phenotypes <- pData(gds48558)
dim(gds48558)

Features  Samples 
   32321       67 


### Normalizing the Data

In [6]:
%%R
print(min(expr))
print(max(expr))

[1] 1.611473
[1] 13.76154


In [7]:
%%R

# log2 transform
qx <- as.numeric(quantile(expr, c(0., 0.25, 0.5, 0.75, 0.99, 1.0), na.rm=T))
LogC <- (qx[5] > 100) || (qx[6]-qx[1] > 50 && qx[2] > 0)
if (LogC) { 
    expr[which(expr <= 0)] <- NaN
    expr <- log2(expr) 
}

dim(expr)

[1] 32321    67


## Differential Expression Analysis

In [8]:
%%R

design <- model.matrix(~group + 0, gds48558)
colnames(design) <- levels(gs)

fit <- lmFit(gds48558, design)  # fit linear model

# set up contrasts of interest and recalculate model coefficients
cts <- paste(groups[1], groups[2], sep="-")
cont.matrix <- makeContrasts(contrasts=cts, levels=design)
fit2 <- contrasts.fit(fit, cont.matrix)

# compute statistics and table of top significant genes
fit2 <- eBayes(fit2, 0.01)
tT <- topTable(fit2, adjust="fdr", sort.by="B", number=Inf)

tT <- subset(tT, select=c("Gene.symbol", "Gene.ID","adj.P.Val","logFC"))
write.table(tT, file='AML_Healthy.tsv', row.names=F, sep="\t", quote=F)

head(tT)

        Gene.symbol Gene.ID    adj.P.Val     logFC
8016932         MPO    4353 3.617813e-19 -5.563501
7970737        FLT3    2322 4.835716e-19 -5.250065
7989647    KIAA0101    9768 6.308160e-19 -4.559135
7982663       BUB1B     701 1.664043e-18 -2.756554
8083422      SUCNR1   56670 1.938573e-18 -2.996816
7926259       MCM10   55388 3.712137e-18 -2.318848


In [9]:
%%R

result <- read.table(file = 'AML_Healthy.tsv', sep = '\t', header = TRUE)
aml.up <- subset(result, logFC > 1 & adj.P.Val < 0.05)
# aml.up.gene <- unique(aml.up$Gene.symbol)
# aml.up.gene <- sub("///.*", "", aml.up.gene)
aml.up.gene <- unique(as.character(strsplit2(aml.up$Gene.symbol, "///")))
write.table(aml.up.gene, file='AML_Healthy_Up.tsv', row.names=F, col.names=F, quote=F)

aml.down <- subset(result, logFC < -1 & adj.P.Val < 0.05)
aml.down.gene <- unique(as.character(strsplit2(aml.down$Gene.symbol, "///")))
write.table(aml.down.gene, file='AML_Healthy_Down.tsv', row.names=F, col.names=F, quote=F)

head(aml.up.gene)

[1] "STK38" "CBX7"  "PLCL2" "PECR"  ""      "HLA-F"
